In [2]:
from pyspark.sql import SparkSession

In [4]:
spark=SparkSession.builder.appName("project").getOrCreate()

Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
25/08/12 17:52:05 WARN Utils: Your hostname, Omars-MacBook-Pro.local, resolves to a loopback address: 127.0.0.1; using 192.168.1.44 instead (on interface en0)
25/08/12 17:52:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j2-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/08/12 17:52:36 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
25/08/12 17:52:36 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [12]:
df=spark.read.csv('sentiment_analysis.csv',header=True)

In [14]:
df.show()

+---+-----+--------------------+
| id|label|               tweet|
+---+-----+--------------------+
|  1|    0|#fingerprint #Pre...|
|  2|    0|Finally a transpa...|
|  3|    0|We love this! Wou...|
|  4|    0|I'm wired I know ...|
|  5|    1|What amazing serv...|
|  6|    1|iPhone software u...|
|  7|    0|Happy for us .. #...|
|  8|    0|New Type C charge...|
|  9|    0|Bout to go shoppi...|
| 10|    0|Photo: #fun #self...|
| 11|    1|hey #apple when y...|
| 12|    1|Ha! Not heavy mac...|
| 13|    1|Contemplating giv...|
| 14|    0|I just made anoth...|
| 15|    1|@shaqlockholmes @...|
| 16|    0|From #DeepEllum t...|
| 17|    0|Like and Share if...|
| 18|    0|Go crazy !! #ipho...|
| 19|    1|The reason I don'...|
| 20|    1|How is the apple ...|
+---+-----+--------------------+
only showing top 20 rows


In [16]:
from pyspark.sql.functions import col

In [18]:
df.select("label").distinct().show()

+-----+
|label|
+-----+
|    0|
|    1|
+-----+



In [20]:
from pyspark.sql.functions import col

# Count tweets that are NOT null or empty
df.filter(col("tweet").isNotNull() & (col("tweet") != "")).count()

7920

In [22]:
from pyspark.sql.functions import col, when, count

# Count how many rows have null or empty tweets
df.select(
    count(when(col("tweet").isNull() | (col("tweet") == ""), "tweet")).alias("null_or_empty_tweets")
).show()


+--------------------+
|null_or_empty_tweets|
+--------------------+
|                   0|
+--------------------+



In [89]:
from pyspark.sql.functions import col, lower, regexp_replace,length,trim
from pyspark.ml.feature import Tokenizer, StopWordsRemover, HashingTF, IDF, StringIndexer,Normalizer


In [26]:
df=df.withColumn('length',length(df['tweet']))

In [28]:
df.show()

+---+-----+--------------------+------+
| id|label|               tweet|length|
+---+-----+--------------------+------+
|  1|    0|#fingerprint #Pre...|   128|
|  2|    0|Finally a transpa...|   131|
|  3|    0|We love this! Wou...|   123|
|  4|    0|I'm wired I know ...|   112|
|  5|    1|What amazing serv...|   124|
|  6|    1|iPhone software u...|    65|
|  7|    0|Happy for us .. #...|   100|
|  8|    0|New Type C charge...|   259|
|  9|    0|Bout to go shoppi...|   130|
| 10|    0|Photo: #fun #self...|   128|
| 11|    1|hey #apple when y...|   124|
| 12|    1|Ha! Not heavy mac...|   122|
| 13|    1|Contemplating giv...|   116|
| 14|    0|I just made anoth...|   133|
| 15|    1|@shaqlockholmes @...|   127|
| 16|    0|From #DeepEllum t...|   123|
| 17|    0|Like and Share if...|   274|
| 18|    0|Go crazy !! #ipho...|   121|
| 19|    1|The reason I don'...|   134|
| 20|    1|How is the apple ...|   115|
+---+-----+--------------------+------+
only showing top 20 rows


In [30]:
df.groupBy('label').mean('length').show()

+-----+------------------+
|label|       avg(length)|
+-----+------------------+
|    0|137.04546996946047|
|    1|104.65597235932873|
+-----+------------------+



In [32]:
from pyspark.sql.functions import col, count

duplicates = df.groupBy("tweet") \
    .agg(count("tweet").alias("count")) \
    .filter(col("count") > 1) \
    .orderBy(col("count").desc())

duplicates.show()


+--------------------+-----+
|               tweet|count|
+--------------------+-----+
|@architecture_3de...|    3|
+--------------------+-----+



In [56]:
df = df.withColumn(
    "tweet",
    lower(regexp_replace("tweet_clean", r"http\S+|@\w+|#\w+|[^a-zA-Z\s]", ""))
)

In [58]:
df = df.withColumn("tweet_clean", regexp_replace("tweet", r"\bpictwitter\w+\b", ""))

In [40]:
from pyspark.sql.functions import regexp_replace, trim

df = df.withColumn("tweet_clean", 
                        trim(regexp_replace("tweet", r"#|\s+", " ")))

In [42]:
from pyspark.sql.functions import regexp_replace

df = df.withColumn("tweet_clean", regexp_replace("tweet", r"\s{2,}", " "))


In [60]:
df.show()

+---+-----+--------------------+------+--------------------+--------------------+
| id|label|               tweet|length|         tweet_clean|               words|
+---+-----+--------------------+------+--------------------+--------------------+
|  1|    0|                test|   128|                test|              [test]|
|  2|    0|finally a transpa...|   131|finally a transpa...|[finally, a, tran...|
|  3|    0|we love this woul...|   123|we love this woul...|[we, love, this, ...|
|  4|    0|im wired i know i...|   112|im wired i know i...|[im, wired, i, kn...|
|  5|    1|what amazing serv...|   124|what amazing serv...|[what, amazing, s...|
|  6|    1|iphone software u...|    65|iphone software u...|[iphone, software...|
|  7|    0|        happy for us|   100|        happy for us|    [happy, for, us]|
|  8|    0|new type c charge...|   259|new type c charge...|[new, type, c, ch...|
|  9|    0|bout to go shoppi...|   130|bout to go shoppi...|[bout, to, go, sh...|
| 10|    0|     

In [52]:
tokenizer = Tokenizer(inputCol='tweet', outputCol='words')
df = tokenizer.transform(df)

In [62]:
df.show()

+---+-----+--------------------+------+--------------------+--------------------+
| id|label|               tweet|length|         tweet_clean|               words|
+---+-----+--------------------+------+--------------------+--------------------+
|  1|    0|                test|   128|                test|              [test]|
|  2|    0|finally a transpa...|   131|finally a transpa...|[finally, a, tran...|
|  3|    0|we love this woul...|   123|we love this woul...|[we, love, this, ...|
|  4|    0|im wired i know i...|   112|im wired i know i...|[im, wired, i, kn...|
|  5|    1|what amazing serv...|   124|what amazing serv...|[what, amazing, s...|
|  6|    1|iphone software u...|    65|iphone software u...|[iphone, software...|
|  7|    0|        happy for us|   100|        happy for us|    [happy, for, us]|
|  8|    0|new type c charge...|   259|new type c charge...|[new, type, c, ch...|
|  9|    0|bout to go shoppi...|   130|bout to go shoppi...|[bout, to, go, sh...|
| 10|    0|     

In [66]:
df.select("tweet").show(10,truncate=False)

+------------------------------------------------------------------------------------------------------------------+
|tweet                                                                                                             |
+------------------------------------------------------------------------------------------------------------------+
|test                                                                                                              |
|finally a transparant silicon case thanks to my uncle                                                             |
|we love this would you go                                                                                         |
|im wired i know im george i was made that way                                                                     |
|what amazing service apple wont even talk to me about a question i have unless i pay them for their stupid support|
|iphone software update fucked up my phone big time stupid iphon

In [68]:
from pyspark.ml.feature import StopWordsRemover
remover = StopWordsRemover(inputCol="words", outputCol="filtered_words")
df= remover.transform(df)


25/08/12 18:09:30 WARN StopWordsRemover: Default locale set was [en_EG]; however, it was not found in available locales in JVM, falling back to en_US locale. Set param `locale` in order to respect another locale.


In [70]:
df.show()

+---+-----+--------------------+------+--------------------+--------------------+--------------------+
| id|label|               tweet|length|         tweet_clean|               words|      filtered_words|
+---+-----+--------------------+------+--------------------+--------------------+--------------------+
|  1|    0|                test|   128|                test|              [test]|              [test]|
|  2|    0|finally a transpa...|   131|finally a transpa...|[finally, a, tran...|[finally, transpa...|
|  3|    0|we love this woul...|   123|we love this woul...|[we, love, this, ...|          [love, go]|
|  4|    0|im wired i know i...|   112|im wired i know i...|[im, wired, i, kn...|[im, wired, know,...|
|  5|    1|what amazing serv...|   124|what amazing serv...|[what, amazing, s...|[amazing, service...|
|  6|    1|iphone software u...|    65|iphone software u...|[iphone, software...|[iphone, software...|
|  7|    0|        happy for us|   100|        happy for us|    [happy, f

In [74]:
df.select('tweet','filtered_words').show()

+--------------------+--------------------+
|               tweet|      filtered_words|
+--------------------+--------------------+
|                test|              [test]|
|finally a transpa...|[finally, transpa...|
|we love this woul...|          [love, go]|
|im wired i know i...|[im, wired, know,...|
|what amazing serv...|[amazing, service...|
|iphone software u...|[iphone, software...|
|        happy for us|         [happy, us]|
|new type c charge...|[new, type, c, ch...|
|bout to go shoppi...|[bout, go, shoppi...|
|               photo|             [photo]|
|hey when you make...|[hey, , make, new...|
|ha not heavy mach...|[ha, heavy, machi...|
|contemplating giv...|[contemplating, g...|
|i just made anoth...|[made, another, c...|
|the battery is so...|[battery, painful...|
|        from towards|         [, towards]|
|like and share if...|[like, share, wan...|
|            go crazy|         [go, crazy]|
|the reason i dont...|[reason, dont, on...|
|how is the apple ...|[apple, st

In [80]:
df.show()

+---+-----+--------------------+------+--------------------+--------------------+--------------------+
| id|label|               tweet|length|         tweet_clean|               words|      filtered_words|
+---+-----+--------------------+------+--------------------+--------------------+--------------------+
|  1|    0|                test|   128|                test|              [test]|              [test]|
|  2|    0|finally a transpa...|   131|finally a transpa...|[finally, a, tran...|[finally, transpa...|
|  3|    0|we love this woul...|   123|we love this woul...|[we, love, this, ...|          [love, go]|
|  4|    0|im wired i know i...|   112|im wired i know i...|[im, wired, i, kn...|[im, wired, know,...|
|  5|    1|what amazing serv...|   124|what amazing serv...|[what, amazing, s...|[amazing, service...|
|  6|    1|iphone software u...|    65|iphone software u...|[iphone, software...|[iphone, software...|
|  7|    0|        happy for us|   100|        happy for us|    [happy, f

In [82]:
from pyspark.sql.functions import size

df = df.withColumn("word_count", size("filtered_words"))

df.select("filtered_words", "word_count").show(truncate=False)


+---------------------------------------------------------------------------------------------------------+----------+
|filtered_words                                                                                           |word_count|
+---------------------------------------------------------------------------------------------------------+----------+
|[test]                                                                                                   |1         |
|[finally, transparant, silicon, case, thanks, uncle]                                                     |6         |
|[love, go]                                                                                               |2         |
|[im, wired, know, im, george, made, way]                                                                 |7         |
|[amazing, service, apple, wont, even, talk, question, unless, pay, stupid, support]                      |11        |
|[iphone, software, update, fucked, phone, big, 

In [93]:
from pyspark.sql.functions import explode, col

# Explode filtered words into separate rows
words_df = df.select(explode(col("filtered_words")).alias("word"))

# Count and sort by frequency
word_counts = words_df.groupBy("word").count().orderBy(col("count").desc())

word_counts.show(40, truncate=False)


+---------+-----+
|word     |count|
+---------+-----+
|new      |876  |
|iphone   |759  |
|phone    |735  |
|apple    |686  |
|follow   |544  |
|love     |414  |
|rt       |321  |
|day      |312  |
|gain     |309  |
|like     |309  |
|im       |308  |
|get      |293  |
|back     |285  |
|got      |279  |
|happy    |239  |
|one      |204  |
|time     |200  |
|photo    |198  |
|cant     |188  |
|everyone |179  |
|make     |171  |
|dont     |171  |
|must     |168  |
|finally  |165  |
|followers|152  |
|app      |150  |
|update   |148  |
|rts      |147  |
|ipod     |142  |
|work     |142  |
|itunes   |139  |
|today    |138  |
|want     |135  |
|case     |135  |
|thanks   |131  |
|ios      |127  |
|good     |126  |
|every    |125  |
|fucking  |125  |
|ipad     |125  |
+---------+-----+
only showing top 40 rows


In [95]:
hashingTF = HashingTF(inputCol="filtered_words", outputCol="hashing")
df = hashingTF.transform(df)

In [97]:
df.show()

+---+-----+--------------------+------+--------------------+--------------------+--------------------+----------+--------------------+
| id|label|               tweet|length|         tweet_clean|               words|      filtered_words|word_count|             hashing|
+---+-----+--------------------+------+--------------------+--------------------+--------------------+----------+--------------------+
|  1|    0|                test|   128|                test|              [test]|              [test]|         1|(262144,[188762],...|
|  2|    0|finally a transpa...|   131|finally a transpa...|[finally, a, tran...|[finally, transpa...|         6|(262144,[7625,143...|
|  3|    0|we love this woul...|   123|we love this woul...|[we, love, this, ...|          [love, go]|         2|(262144,[148675,1...|
|  4|    0|im wired i know i...|   112|im wired i know i...|[im, wired, i, kn...|[im, wired, know,...|         7|(262144,[31015,51...|
|  5|    1|what amazing serv...|   124|what amazing ser

In [109]:
idf=IDF(inputCol='hashing',outputCol='feature')

In [111]:
idf_model=idf.fit(df)

In [113]:
rescale=idf_model.transform(df)

In [115]:
rescale.select('label','feature').show()

+-----+--------------------+
|label|             feature|
+-----+--------------------+
|    0|(262144,[188762],...|
|    0|(262144,[7625,143...|
|    0|(262144,[148675,1...|
|    0|(262144,[31015,51...|
|    1|(262144,[1696,437...|
|    1|(262144,[1696,612...|
|    0|(262144,[109156,1...|
|    0|(262144,[36225,49...|
|    0|(262144,[59009,10...|
|    0|(262144,[82583],[...|
|    1|(262144,[12710,61...|
|    1|(262144,[1797,254...|
|    1|(262144,[31732,79...|
|    0|(262144,[2437,148...|
|    1|(262144,[11275,48...|
|    0|(262144,[79497],[...|
|    0|(262144,[6122,317...|
|    0|(262144,[113024,1...|
|    1|(262144,[21823,87...|
|    1|(262144,[41748,74...|
+-----+--------------------+
only showing top 20 rows


25/08/12 19:47:30 WARN DAGScheduler: Broadcasting large task binary with size 4.0 MiB


In [129]:
from pyspark.ml.classification import NaiveBayes 
from pyspark.ml import Pipeline

In [131]:
nb=NaiveBayes()